<h1 align='center'> <font color="red"> Bagging </font> </h1>

<h2 align="center"> Resampling with replacement </h2>

**Resampling with replacement involves taking the original sample and using a random number generator to pick a number from the dataset. This number is then kept in the dataset so that it can be chosen again, and a new number is randomly selected. This process is repeated until a new sample of size n is created.**

**Each new individual sample generated has contain same sample multiple times**

<img src="https://i.postimg.cc/qv5yBYKD/ensem.png">

**Here M1,M2,M3 are models trained using any ml algorithm, and output of each model to a input is taken and select majority among them and it is the final output.**

**This technique is also called " Bootstrap - Aggregation "**

**bootstrap - random sample creation  | Aggregation - majority vote among model | Random forest is a bagging Technique**

<img src="https://i.postimg.cc/htDqsD26/desc.png">
<h3 align="center">This is how random data picking</h3>

### Bagged Tree - Underlying models be Tree

### Bagging - Underlying models be any such as svm, logistic regression, knn

**We will use pima indian diabetes dataset to predict if a person has a diabetes or not based on certain features such as blood pressure, skin thickness, age etc. We will train a standalone model first and then use bagging ensemble technique to check how it can improve the performance of the model**

In [4]:
import pandas as pd

df = pd.read_csv("C:/Users/Dell/OneDrive/Documents/machine_learning models/diabetes.csv")
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
df.Outcome.value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

**There is slight imbalance in our dataset**

# Train-Test Split

In [8]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

### Stratify keyword ensure that test and train data has equal no. of samples

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

# Train using stand alone model

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.68181818, 0.67532468, 0.7012987 , 0.77777778, 0.71895425])

In [12]:
scores.mean()

0.7110347169170699

In [18]:
import warnings
warnings.filterwarnings("ignore")

# Train using Bagging

In [19]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.7534722222222222

In [20]:
bag_model.score(X_test, y_test)

0.7760416666666666

In [21]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

**We can see some improvement in test score with bagging classifier as compared to a standalone classifier**

In [22]:
scores.mean()

0.7578728461081402

# Train using Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

0.7774297597827009